In [1]:
import pandas as pd
import numpy as np

Зарузим датафрейм и изучим его поподробнее.

In [2]:
df = pd.read_csv("crx.data", header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64
15    690 non-null object
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [5]:
df[15].value_counts()

-    383
+    307
Name: 15, dtype: int64

Если внимательнее изучить данные, то можно обнаружить, что в них присутствуют значения, которые представлены, как "**?**". Судя по всему, это пропуски в данных.

In [6]:
df.isin(['?']).sum(axis=0)

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

Заменим все "**?**" на **NaN** при помощи метода **replace**, а затем для нечисловых признаков произведем замену на модальные значения, а для числовых – на средние, используя метод **fillna**.

In [7]:
df.replace('?', np.nan, inplace=True)
df[1] = df[1].astype('float')
df.fillna(df.mean(), inplace=True)
df.fillna(df.mode().iloc[0], inplace=True)

Теперь нам необходимо преобразовать нечисловые признаки и целевую переменную в числовой вид.

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
X,y = df.iloc[:,0:15] , df.iloc[:,15]
y = LabelEncoder().fit_transform(y)
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = OrdinalEncoder().fit_transform(X[col].values.reshape(-1, 1))

Разделим нашу выборку на тестовую и тренировочную.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

Обучение моделей

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

1) Логистическая регрессия

In [11]:
logreg_pipe = make_pipeline(MinMaxScaler(feature_range=(0, 1)), LogisticRegression(random_state=42))
logreg_params_grid = {'logisticregression__C': [0.01, 0.001, 0.0001], 
                      'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear'],
                      'logisticregression__class_weight': ['balanced', None]}
logreg_grid = GridSearchCV(logreg_pipe, logreg_params_grid, scoring='f1')  
logreg_grid.fit(X_train, y_train)
print('Лучшие параметры:', logreg_grid.best_params_)
print('F-мера на перекрестной проверке:', logreg_grid.best_score_)
print('F-мера логистической регрессии на тестовом наборе:', logreg_grid.score(X_test, y_test))

Лучшие параметры: {'logisticregression__C': 0.01, 'logisticregression__class_weight': 'balanced', 'logisticregression__solver': 'newton-cg'}
F-мера на перекрестной проверке: 0.8536395924094758
F-мера логистической регрессии на тестовом наборе: 0.8773946360153256


2) Дерево решений

In [12]:
dtc_pipe = make_pipeline(MinMaxScaler(feature_range=(0, 1)), DecisionTreeClassifier(random_state=42))
dtc_params_grid = {'decisiontreeclassifier__min_samples_split': range(2, 200, 5), 
                      'decisiontreeclassifier__criterion': ['gini', 'entropy'],
                      'decisiontreeclassifier__class_weight': ['balanced', None],
                      'decisiontreeclassifier__max_features': ['auto',None,'log2']}
dtc_grid = GridSearchCV(dtc_pipe, dtc_params_grid, scoring='f1')  
dtc_grid.fit(X_train, y_train)
print('Лучшие параметры:', dtc_grid.best_params_)
print('F-мера на перекрестной проверке:', dtc_grid.best_score_)
print('F-мера дерева решений на тестовом наборе:', dtc_grid.score(X_test, y_test))

Лучшие параметры: {'decisiontreeclassifier__class_weight': None, 'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassifier__max_features': None, 'decisiontreeclassifier__min_samples_split': 32}
F-мера на перекрестной проверке: 0.8573349191665566
F-мера дерева решений на тестовом наборе: 0.81203007518797


3) Случайный лес

In [13]:
rfc_pipe = make_pipeline(MinMaxScaler(feature_range=(0, 1)), RandomForestClassifier(random_state=42))
rfc_params_grid = {'randomforestclassifier__min_samples_split': range(2, 200, 5), 
                   'randomforestclassifier__criterion': ['gini', 'entropy'],
                   'randomforestclassifier__class_weight': ['balanced', None],
                   'randomforestclassifier__max_features': ['auto', None, 'log2']}
rfc_grid = GridSearchCV(rfc_pipe, rfc_params_grid, scoring='f1')  
rfc_grid.fit(X_train, y_train)
print('Лучшие параметры:', rfc_grid.best_params_)
print('F-мера на перекрестной проверке:', rfc_grid.best_score_)
print('F-мера случайного леса на тестовом наборе:', rfc_grid.score(X_test, y_test))

Лучшие параметры: {'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__min_samples_split': 12}
F-мера на перекрестной проверке: 0.8888481814389199
F-мера случайного леса на тестовом наборе: 0.8808664259927799


4) Метод опорных векторов

In [14]:
svc_pipe = make_pipeline(MinMaxScaler(feature_range=(0, 1)),svm.SVC(random_state=42))
svc_params_grid = {'svc__C': [1, 5, 10, 15], 
                   'svc__gamma': ['scale', 'auto'],
                   'svc__class_weight': ['balanced', None]}
svc_grid = GridSearchCV(svc_pipe, svc_params_grid, scoring='f1')  
svc_grid.fit(X_train, y_train)
print('Лучшие параметры:', svc_grid.best_params_)
print('F-мера на перекрестной проверке:', svc_grid.best_score_)
print('F-мера метода опорных векторов на тестовом наборе:', svc_grid.score(X_test, y_test))

Лучшие параметры: {'svc__C': 1, 'svc__class_weight': None, 'svc__gamma': 'scale'}
F-мера на перекрестной проверке: 0.8509320136679628
F-мера метода опорных векторов на тестовом наборе: 0.8769230769230769
